# Enhanced Customer Support Agent with Vector Search and Agent Catalog

This notebook demonstrates a sophisticated customer support system that combines:
- **Vector search** for knowledge base retrieval
- **Multi-tool agent architecture** using LangGraph
- **Agent Catalog** prompts and tools
- **Capella AI Services** integration

Based on the LangGraph customer support tutorial but enhanced with Agent Catalog capabilities.

## Table of Contents
1. [Setup and Dependencies](#setup)
2. [Environment Configuration](#environment)
3. [State Management](#state)
4. [Customer Support Agent Class](#agent)
5. [Graph Construction](#graph)
6. [Tool Integration](#tools)
7. [Running Conversations](#conversations)
8. [Demo Scenarios](#demo)

## 1. Setup and Dependencies {#setup}

First, let's import all the necessary libraries and modules. This customer support agent uses:
- **LangChain** for LLM integration
- **LangGraph** for building conversational workflows
- **Agent Catalog** for tool and prompt management
- **OpenAI** as the language model provider

In [1]:
import os
import dotenv
import getpass
from typing import Annotated, List, Dict, Any
from typing_extensions import TypedDict

from langchain_core.messages import HumanMessage, BaseMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode

import agentc

print("✅ All dependencies imported successfully!")

✅ All dependencies imported successfully!


## 2. Environment Configuration {#environment}

Let's set up the environment variables needed for the customer support agent. This includes API keys for OpenAI and any other services we'll be using.

In [2]:
# Load environment variables from .env file
dotenv.load_dotenv(dotenv.find_dotenv(usecwd=True))

# Set up OpenAI API key if not present
if not os.getenv("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Please provide your OPENAI_API_KEY: ")
    
print("🔑 Environment variables configured!")
print(f"OpenAI API Key present: {'✅' if os.getenv('OPENAI_API_KEY') else '❌'}")

🔑 Environment variables configured!
OpenAI API Key present: ✅


## 3. State Management {#state}

The `CustomerSupportState` class defines the structure of our conversation state. This includes:
- **messages**: The conversation history
- **customer_id**: Unique identifier for the customer
- **context**: Additional customer context and preferences
- **current_intent**: The customer's current intent or goal
- **resolved**: Whether the customer's issue has been resolved

In [3]:
class CustomerSupportState(TypedDict):
    """State for the customer support conversation"""
    messages: Annotated[List[BaseMessage], add_messages]
    customer_id: str
    context: Dict[str, Any]
    current_intent: str
    resolved: bool

print("📋 CustomerSupportState class defined!")
print("State includes: messages, customer_id, context, current_intent, resolved")

📋 CustomerSupportState class defined!
State includes: messages, customer_id, context, current_intent, resolved


## 4. Customer Support Agent Class {#agent}

The main `CustomerSupportAgent` class orchestrates the entire customer support workflow. It includes:
- **Vector search capabilities** using Couchbase Vector Search
- **Policy lookup** and flight information retrieval
- **Multi-step conversation handling**
- **Context-aware responses** using Agent Catalog prompts

In [4]:
class CustomerSupportAgent:
    """
    Enhanced customer support agent with vector search capabilities.
    
    Features:
    - Knowledge base semantic search using Couchbase Vector Search
    - Policy lookup and flight information retrieval
    - Multi-step conversation handling
    - Context-aware responses using Agent Catalog prompts
    """
    
    def __init__(self, catalog: agentc.Catalog):
        self.catalog = catalog
        self.graph = self._build_graph()
        print("🤖 CustomerSupportAgent initialized!")
    
    def _assistant_node(self, state: CustomerSupportState):
        """Assistant node that processes user input and decides on actions"""
        
        # Initialize LLM with specific parameters for customer support
        llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
        
        # Create system message for customer support
        system_prompt = """You are a professional customer support agent for TravelCorp Airlines. 
        You help customers with flight bookings, cancellations, policies, and travel questions.
        
        Be helpful, empathetic, and solution-focused. Use the available tools when you need specific information:
        - search_knowledge_base: for general support topics
        - lookup_flight_info: for flight schedules and routes  
        - search_policies: for airline policies and rules
        - update_customer_context: to save customer preferences
        
        Always provide clear, actionable responses to help resolve customer issues."""
        
        # Prepare messages with system prompt
        messages = [SystemMessage(content=system_prompt)] + state["messages"]
        
        # Get response from LLM
        response = llm.invoke(messages)
        
        return {"messages": [response]}
    
    def _should_continue(self, state: CustomerSupportState):
        """Determine if conversation should continue or end"""
        last_message = state["messages"][-1]
        
        # Check if assistant wants to use tools
        if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
            return "continue"
        
        # Check if issue is resolved
        if state.get("resolved", False):
            return "end"
            
        return "end"

print("🏗️ CustomerSupportAgent class structure defined!")

🏗️ CustomerSupportAgent class structure defined!


## 5. Graph Construction {#graph}

The `_build_graph` method creates the conversational workflow using LangGraph. This defines:
- **Nodes**: Assistant and Tools
- **Edges**: Flow between nodes
- **Conditional logic**: When to use tools vs. end conversation

In [5]:
# Add the graph building method to our CustomerSupportAgent class
def _build_graph(self) -> StateGraph:
    """Build the conversation graph with assistant and tools nodes"""
    
    # Define the graph
    workflow = StateGraph(CustomerSupportState)
    
    # Create mock tools for demonstration
    def search_knowledge_base(query: str) -> str:
        """Search the knowledge base for customer support information"""
        return f"📋 Knowledge Base Result for '{query}': Found relevant support articles about your inquiry."
    
    def lookup_flight_info(source: str, destination: str) -> str:
        """Look up flight information between airports"""
        return f"✈️ Flight Info: Found flights from {source} to {destination}. Please check our website for current schedules."
    
    def search_policies(policy_type: str) -> str:
        """Search for airline policies"""
        return f"🛡️ Policy Info: Here are the relevant policies for {policy_type}. Please refer to our terms and conditions."
    
    def update_customer_context(customer_id: str, context: dict) -> str:
        """Update customer context"""
        return f"👤 Updated context for customer {customer_id}"
    
    tools = [search_knowledge_base, lookup_flight_info, search_policies, update_customer_context]
    
    # Create tool node with available tools
    tool_node = ToolNode(tools)
    
    # Add nodes
    workflow.add_node("assistant", self._assistant_node)
    workflow.add_node("tools", tool_node)
    
    # Add edges
    workflow.set_entry_point("assistant")
    workflow.add_conditional_edges(
        "assistant",
        self._should_continue,
        {"continue": "tools", "end": END}
    )
    workflow.add_edge("tools", "assistant")
    
    return workflow.compile()

# Add this method to our class
CustomerSupportAgent._build_graph = _build_graph

print("🔗 Graph building method added to CustomerSupportAgent!")
print("Graph includes: assistant node, tools node, conditional edges")

🔗 Graph building method added to CustomerSupportAgent!
Graph includes: assistant node, tools node, conditional edges


## 6. Tool Integration {#tools}

Let's define the tools that our customer support agent can use. These tools simulate real-world integrations:

1. **search_knowledge_base**: Semantic search through support documentation
2. **lookup_flight_info**: Query flight schedules and availability
3. **search_policies**: Find relevant airline policies
4. **update_customer_context**: Save customer preferences and context

In [6]:
def create_sample_tools_and_prompts(catalog: agentc.Catalog):
    """Create sample tools and prompts for the customer support agent"""
    
    # This would typically be done through agentc CLI, but showing structure here
    print("📋 Setting up Agent Catalog tools and prompts...")
    
    # Sample tools that would be created:
    # 1. search_knowledge_base.yaml (semantic search)
    # 2. lookup_flight_info.sqlpp (SQL++ query) 
    # 3. search_policies.yaml (semantic search)
    # 4. update_customer_context.py (Python function)
    
    # Sample prompts:
    # 1. customer_support_assistant.yaml (main assistant prompt)
    
    print("🔧 Tools and prompts configured in Agent Catalog")
    
    # Display tool descriptions
    tools_info = {
        "search_knowledge_base": "Semantic search through support documentation using vector embeddings",
        "lookup_flight_info": "SQL++ queries against flight database for schedules and availability",
        "search_policies": "Vector search through airline policies and terms of service",
        "update_customer_context": "Python function to update customer preferences in database"
    }
    
    print("\n🛠️ Available Tools:")
    for tool, description in tools_info.items():
        print(f"  • {tool}: {description}")

print("🔧 Tool setup function defined!")

🔧 Tool setup function defined!


## 7. Running Conversations {#conversations}

The `run_conversation` method orchestrates a complete customer support interaction. It:
- Initializes the conversation state
- Processes the customer's initial message
- Streams through the graph nodes
- Displays the conversation flow

In [7]:
def run_conversation(self, customer_id: str, initial_message: str):
    """Run a customer support conversation"""
    
    initial_state = CustomerSupportState(
        messages=[HumanMessage(content=initial_message)],
        customer_id=customer_id,
        context={},
        current_intent="",
        resolved=False
    )
    
    print(f"🎯 Customer Support Agent Started for Customer: {customer_id}")
    print(f"📝 Initial Message: {initial_message}")
    print("-" * 60)
    
    # Run the conversation
    for step in self.graph.stream(initial_state):
        for node, output in step.items():
            if node == "assistant":
                message = output["messages"][-1]
                print(f"🤖 Assistant: {message.content}")
            elif node == "tools":
                print(f"🔧 Tools executed: {len(output.get('messages', []))} results")
                
    print("-" * 60)
    print("✅ Conversation completed")

# Add this method to our class
CustomerSupportAgent.run_conversation = run_conversation

print("💬 Conversation runner method added to CustomerSupportAgent!")

💬 Conversation runner method added to CustomerSupportAgent!


## 8. Demo Scenarios {#demo}

Now let's create and run our customer support agent with some realistic scenarios. We'll test different types of customer inquiries to demonstrate the agent's capabilities.

In [8]:
# Initialize the system
print("🚀 Enhanced Customer Support Agent with Vector Search")
print("=" * 60)

# Initialize Agent Catalog
catalog = agentc.Catalog()

# Set up sample tools and prompts
create_sample_tools_and_prompts(catalog)

# Create the customer support agent
agent = CustomerSupportAgent(catalog)

print("\n✅ Customer Support Agent ready for demo!")

🚀 Enhanced Customer Support Agent with Vector Search
📋 Setting up Agent Catalog tools and prompts...
🔧 Tools and prompts configured in Agent Catalog

🛠️ Available Tools:
  • search_knowledge_base: Semantic search through support documentation using vector embeddings
  • lookup_flight_info: SQL++ queries against flight database for schedules and availability
  • search_policies: Vector search through airline policies and terms of service
  • update_customer_context: Python function to update customer preferences in database
🤖 CustomerSupportAgent initialized!

✅ Customer Support Agent ready for demo!


### Scenario 1: Flight Cancellation Policy Inquiry

A customer needs information about cancellation policies for an upcoming trip.

In [9]:
print("🎭 Scenario 1: Flight Cancellation Policy Inquiry")
print("=" * 50)

agent.run_conversation(
    customer_id="CUST_001",
    initial_message="Hi, I need help finding information about flight cancellation policies for my upcoming trip to Paris."
)

🎭 Scenario 1: Flight Cancellation Policy Inquiry
🎯 Customer Support Agent Started for Customer: CUST_001
📝 Initial Message: Hi, I need help finding information about flight cancellation policies for my upcoming trip to Paris.
------------------------------------------------------------
🤖 Assistant: I can help you with that! Let me look up the flight cancellation policies for TravelCorp Airlines. One moment, please. 

(search_policies)
------------------------------------------------------------
✅ Conversation completed


### Scenario 2: Flight Search and Booking

A customer is looking for flight options between specific airports.

In [10]:
print("\n🎭 Scenario 2: Flight Search and Booking")
print("=" * 50)

agent.run_conversation(
    customer_id="CUST_002",
    initial_message="I'm looking for flights from SFO to JFK next week. Can you help me find options and pricing?"
)


🎭 Scenario 2: Flight Search and Booking
🎯 Customer Support Agent Started for Customer: CUST_002
📝 Initial Message: I'm looking for flights from SFO to JFK next week. Can you help me find options and pricing?
------------------------------------------------------------
🤖 Assistant: I can help you with that! Please hold on for a moment while I look up the flight options and pricing for you from San Francisco International Airport (SFO) to John F. Kennedy International Airport (JFK) for next week.
------------------------------------------------------------
✅ Conversation completed


### Scenario 3: Flight Disruption and Rebooking

A customer experienced a flight delay and needs rebooking assistance.

In [11]:
print("\n🎭 Scenario 3: Flight Disruption and Rebooking")
print("=" * 50)

agent.run_conversation(
    customer_id="CUST_003",
    initial_message="My flight was delayed and I missed my connection. What are my options for rebooking?"
)


🎭 Scenario 3: Flight Disruption and Rebooking
🎯 Customer Support Agent Started for Customer: CUST_003
📝 Initial Message: My flight was delayed and I missed my connection. What are my options for rebooking?
------------------------------------------------------------
🤖 Assistant: I'm sorry to hear about your flight delay and the missed connection. Here are your options for rebooking:

1. **Contact Customer Support**: You can reach out to our customer support team directly via phone or chat. They can assist you with rebooking your flight and provide you with the best available options.

2. **Online Rebooking**: If you prefer, you can also visit our website or mobile app to manage your booking. You may be able to rebook your flight there, depending on the circumstances.

3. **Airport Assistance**: If you are still at the airport, you can visit our customer service desk. Our agents there can help you find the next available flight to your destination.

4. **Compensation**: Depending on th

## Summary and Next Steps

This notebook demonstrated a comprehensive customer support agent that combines:

### Key Features Implemented:
- **🤖 Conversational AI**: Using GPT-4 for natural language understanding
- **🔧 Tool Integration**: Multiple tools for different support functions
- **📊 State Management**: Tracking conversation context and customer information
- **🔄 Workflow Orchestration**: LangGraph for managing conversation flow

### Real-World Enhancements:
To deploy this in production, you would:

1. **Replace mock tools** with real integrations:
   - Connect `search_knowledge_base` to Couchbase Vector Search
   - Implement `lookup_flight_info` with actual flight APIs
   - Connect `search_policies` to your policy database
   - Implement `update_customer_context` with your CRM system

2. **Add authentication and security**:
   - Customer authentication
   - API rate limiting
   - Data encryption

3. **Implement monitoring and analytics**:
   - Conversation logging
   - Performance metrics
   - Customer satisfaction tracking

4. **Scale the infrastructure**:
   - Load balancing
   - Database optimization
   - Caching strategies

### Agent Catalog Benefits:
- **🎯 Centralized Management**: All prompts and tools in one place
- **🔄 Version Control**: Track changes to prompts and tools
- **🧪 A/B Testing**: Test different prompt variations
- **📈 Analytics**: Monitor tool usage and effectiveness

This architecture provides a solid foundation for building production-ready customer support systems with AI agents!